In [53]:
TARGET='exactly'
TOKEN_TYPE='adv'
FRQ_FLOOR=5
SELECT_SEP = ': '
COMPARE_SEP = '∩'

In [54]:
import pandas as pd 
from pathlib import Path
from am_notebooks import *

from source.utils.associate import AM_DF_DIR, POLAR_DIR, TOP_AM_DIR, ALPHA

ASSETS_PATH=Path('/home/arh234/WinHome/Documents/dissertex/justwriting/assets')
TARGET_OUT=ASSETS_PATH/TARGET

In [55]:
am_dfs = pd.Series(POLAR_DIR.rglob(f'adv/extra/*min{FRQ_FLOOR}x*.parq'))
# for i,ad in enumerate(am_dfs, start=1): 
#     print(f'{i}) {ad.relative_to(AM_DF_DIR)}')
print(am_dfs.to_frame('path').to_markdown(tablefmt='simple_grid') )

┌────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│    │ path                                                                                                               │
├────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│  0 │ /share/compling/projects/sanpi/results/assoc_df/polar/RBdirect/adv/extra/polarized-adv_ALL-direct_min5x_extra.parq │
├────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│  1 │ /share/compling/projects/sanpi/results/assoc_df/polar/RBdirect/adv/extra/polarized-adv_NEQ-direct_min5x_extra.parq │
├────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│  2 │ /share/compling/projects/sanpi/results/assoc_df/polar/mirror/adv/extra/polarized-adv_ALL-mirror_min5x_extra.parq   │
├────┼──

## Load AMs for targeted lexical type and combine different perspectives

In [56]:
target_portions = []
for path in am_dfs:
    target_portions.append(
        update_index(assign_polarity(
            pd.read_parquet(
                path, engine='pyarrow',
                filters=[('l2', '==', TARGET)]
            ))).reset_index().assign(
                selection=path.name.removeprefix(
                    f'polarized-{TOKEN_TYPE}_')[:3] + '+',
                approx=path.parent.parent.parent.name[-6:])
    )

target_am = pd.concat(target_portions)
target_am = target_am.assign(
    # compare_key=('*' + target_am.selection.astype('string')
    #              + '*:' + target_am.key.astype('string'))
    comparison=(target_am.selection.astype('string') + SELECT_SEP
                 + (target_am.key.astype('string')
                    .str.replace('~', COMPARE_SEP)
                    .str.replace(TARGET, f'*{TARGET}*'))
                 )
).set_index('comparison').convert_dtypes()
target_am

negated  positive
False    True        1
True     False       1
Name: count, dtype: int64
negated  positive
False    True        1
True     False       1
Name: count, dtype: int64
negated  positive
False    True        1
True     False       1
Name: count, dtype: int64
negated  positive
False    True        1
True     False       1
Name: count, dtype: int64


,key,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,...,conservative_log_ratio_05,conservative_log_ratio_nc,conservative_log_ratio_dv,f_sqrt,f1_sqrt,f2_sqrt,l1,polarity,selection,approx
comparison,,,,,,,,,,,,,,,,,,,,,
ALL+: COM∩*exactly*,COM~exactly,exactly,12296,53634.44928,-216169.97875,-1.893734,-0.737327,-0.013626,0.219145,0.000179,...,-6.197012,-6.229784,-6.197114,110.88733,8293.834578,236.873384,COMPLEMENT,pos,ALL+,direct
ALL+: NEGany∩*exactly*,NEGany~exactly,exactly,43813,2474.55072,216169.97875,1.893734,0.737327,0.013626,0.780855,0.013805,...,6.197012,6.229784,6.197114,209.315551,1781.482809,236.873384,NEGATED,neg,ALL+,direct
NEQ+: COM∩*exactly*,COM~exactly,exactly,565,22189.0,-55763.99325,-1.895135,-0.490699,-0.013627,0.012732,0.000178,...,-5.995852,-6.107133,-5.988919,23.769729,1781.482809,210.660865,COMPLEMENT,pos,NEQ+,direct
NEQ+: NEGany∩*exactly*,NEGany~exactly,exactly,43813,22189.0,55763.99325,1.895135,0.490699,0.013627,0.987268,0.013805,...,5.995852,6.107133,5.988919,209.315551,1781.482809,210.660865,NEGATED,neg,NEQ+,direct
ALL+: NEGmir∩*exactly*,NEGmir~exactly,exactly,813,178.96756,1868.339071,1.249731,0.615346,0.00263,0.788555,0.002787,...,3.655825,3.842678,3.640831,28.513155,540.124986,32.109189,NEGMIR,neg,ALL+,mirror
ALL+: POS∩*exactly*,POS~exactly,exactly,218,852.03244,-1868.339071,-1.249731,-0.615346,-0.00263,0.211445,0.000157,...,-3.655825,-3.842678,-3.640831,14.764823,1178.515167,32.109189,POSMIR,pos,ALL+,mirror
NEQ+: NEGmir∩*exactly*,NEGmir~exactly,exactly,813,431.0,819.818665,1.216891,0.443811,0.002619,0.943155,0.002787,...,3.16991,3.473963,3.13939,28.513155,540.124986,29.359837,NEGMIR,neg,NEQ+,mirror
NEQ+: POS∩*exactly*,POS~exactly,exactly,49,431.0,-819.818665,-1.216891,-0.443811,-0.002619,0.056845,0.000168,...,-3.16991,-3.473963,-3.13939,7.0,540.124986,29.359837,POSMIR,pos,NEQ+,mirror


In [57]:
all_cols_csv = TARGET_OUT/f'EveryEnv-{TARGET.capitalize()}_am.{timestamp_today()}.csv'
if not all_cols_csv.is_file():
    target_am.to_csv(all_cols_csv)

In [58]:
# target_am.update(target_am.select_dtypes(include='float').apply(pd.to_numeric, downcast='float'))
# lrc_vals = adjust_am_names(target_am.filter(regex=r'conservative|pr?o[lx]|select'))
# lrc_vals = lrc_vals.round(3).rename(columns={'LRC': f'LRC[_{str(ALPHA).removeprefix("0.")}]', 
#                                   'LRC_nc': 'LRC_no-correction', 
#                                   'LRC_dv': 'LRC_default-vocab-size'}).sort_values(['approx', 'selection', 'polarity'])
# nb_show_table(lrc_vals[['approx', 'selection', 'polarity'] + lrc_vals.filter(like='LRC').columns.tolist()], 
#               outpath= TARGET_OUT/f'{TARGET}_LRCvariations.{timestamp_today()}.md', n_dec=3)


|                              | `approx`   | `selection`   | `polarity`   |   `LRC[_005]` |   `LRC_001` |   `LRC_01` |   `LRC_05` |   `LRC_no-correction` |   `LRC_default-vocab-size` |
|:-----------------------------|:-----------|:--------------|:-------------|--------------:|------------:|-----------:|-----------:|----------------------:|---------------------------:|
| **ALL+: NEGany & *exactly*** | direct     | ALL+          | neg          |         6.191 |       6.187 |      6.193 |      6.197 |                 6.230 |                      6.197 |
| **ALL+: COM & *exactly***    | direct     | ALL+          | pos          |        -6.191 |      -6.187 |     -6.193 |     -6.197 |                -6.230 |                     -6.197 |
| **NEQ+: NEGany & *exactly*** | direct     | NEQ+          | neg          |         5.970 |       5.954 |      5.978 |      5.996 |                 6.107 |                      5.989 |
| **NEQ+: COM & *exactly***    | direct     | NEQ+          | pos          |        -5.970 |      -5.954 |     -5.978 |     -5.996 |                -6.107 |                     -5.989 |
| **ALL+: NEGmir & *exactly*** | mirror     | ALL+          | neg          |         3.608 |       3.578 |      3.622 |      3.656 |                 3.843 |                      3.641 |
| **ALL+: POS & *exactly***    | mirror     | ALL+          | pos          |        -3.608 |      -3.578 |     -3.622 |     -3.656 |                -3.843 |                     -3.641 |
| **NEQ+: NEGmir & *exactly*** | mirror     | NEQ+          | neg          |         3.088 |       3.035 |      3.112 |      3.170 |                 3.474 |                      3.139 |
| **NEQ+: POS & *exactly***    | mirror     | NEQ+          | pos          |        -3.088 |      -3.035 |     -3.112 |     -3.170 |                -3.474 |                     -3.139 |


> saved as:  
> `"/home/arh234/WinHome/Documents/dissertex/justwriting/assets/exactly/exactly_LRCvariations.2024-08-09.md"`



In [59]:
focus_cols = FOCUS_DICT['NEQ']['polar']
target_am = adjust_am_names(
    target_am[['approx', 'selection','polarity']].join(target_am.filter(focus_cols + adjust_am_names(focus_cols))))
target_am = target_am.sort_values(['approx', 'selection','polarity'])
target_am.reset_index().set_index(['approx', 'selection', 'polarity']).reset_index().set_index('comparison')

,approx,selection,polarity,f,dP1,LRC,P1,G2,l1,l2,...,f2,N,exp_f,unexp_f,unexp_r,dP2,P2,odds_r_disc,t,MI
comparison,,,,,,,,,,,,,,,,,,,,,
ALL+: NEGany∩*exactly*,direct,ALL+,neg,43813,0.737327,6.190841,0.780855,216169.97875,NEGATED,exactly,...,56109,71961373,2474.55072,41338.44928,0.94352,0.013626,0.013805,1.893734,197.493439,1.248107
ALL+: COM∩*exactly*,direct,ALL+,pos,12296,-0.737327,-6.190841,0.219145,-216169.97875,COMPLEMENT,exactly,...,56109,71961373,53634.44928,-41338.44928,-3.361943,-0.013626,0.000179,-1.893734,-372.796875,-0.63968
NEQ+: NEGany∩*exactly*,direct,NEQ+,neg,43813,0.490699,5.970259,0.987268,55763.99325,NEGATED,exactly,...,44378,6347362,22189.0,21624.0,0.493552,0.013627,0.013805,1.895135,103.308136,0.295465
NEQ+: COM∩*exactly*,direct,NEQ+,pos,565,-0.490699,-5.970259,0.012732,-55763.99325,COMPLEMENT,exactly,...,44378,6347362,22189.0,-21624.0,-38.272568,-0.013627,0.000178,-1.895135,-909.728516,-1.594089
ALL+: NEGmir∩*exactly*,mirror,ALL+,neg,813,0.615346,3.60849,0.788555,1868.339071,NEGMIR,exactly,...,1031,1680633,178.96756,634.03241,0.779868,0.00263,0.002787,1.249731,22.236488,0.657316
ALL+: POS∩*exactly*,mirror,ALL+,pos,218,-0.615346,-3.60849,0.211445,-1868.339071,POSMIR,exactly,...,1031,1680633,852.03244,-634.03241,-2.908406,-0.00263,0.000157,-1.249731,-42.942097,-0.592
NEQ+: NEGmir∩*exactly*,mirror,NEQ+,neg,813,0.443811,3.087807,0.943155,819.818665,NEGMIR,exactly,...,862,583470,431.0,382.0,0.469865,0.002619,0.002787,1.216891,13.397325,0.275613
NEQ+: POS∩*exactly*,mirror,NEQ+,pos,49,-0.443811,-3.087807,0.056845,-819.818665,POSMIR,exactly,...,862,583470,431.0,-382.0,-7.795918,-0.002619,0.000168,-1.216891,-54.57143,-0.944281


In [60]:
nb_show_table(target_am.select_dtypes(include='number'), n_dec=2)


|                            |    `f` |   `dP1` |   `LRC` |   `P1` |        `G2` |       `f1` |   `f2` |        `N` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `odds_r_disc` |     `t` |   `MI` |
|:---------------------------|-------:|--------:|--------:|-------:|------------:|-----------:|-------:|-----------:|----------:|------------:|------------:|--------:|-------:|----------------:|--------:|-------:|
| **ALL+: NEGany∩*exactly*** | 43,813 |    0.74 |    6.19 |   0.78 |  216,169.98 |  3,173,681 | 56,109 | 71,961,373 |  2,474.55 |   41,338.45 |        0.94 |    0.01 |   0.01 |            1.89 |  197.49 |   1.25 |
| **ALL+: COM∩*exactly***    | 12,296 |   -0.74 |   -6.19 |   0.22 | -216,169.98 | 68,787,692 | 56,109 | 71,961,373 | 53,634.45 |  -41,338.45 |       -3.36 |   -0.01 |   0.00 |           -1.89 | -372.80 |  -0.64 |
| **NEQ+: NEGany∩*exactly*** | 43,813 |    0.49 |    5.97 |   0.99 |   55,763.99 |  3,173,681 | 44,378 |  6,347,362 | 22,189.00 |   21,624.00 |

In [61]:
# Output environment overview table
env_info_path = cobble_dated_path(data_dir=TARGET_OUT.parent, suffix='.md',
                                  undated_stem='envs_overview')
if not env_info_path.is_file():
    dir_names = {'NEGATED': '`RBdirect`',
                 'COMPLEMENT': '`not-RBdirect`',
                 'NEGMIR': '`NEGmirror`',
                 'POSMIR': '`POSmirror`'}
    aliases = {'NEGATED': '$*N$; NEGany',
               'COMPLEMENT': '$*C_N$; $*\complement_N$; COM',
               'NEGMIR': '$@N$; NEGmir',
               'POSMIR': '$@P$; POS',
               'neg': '$(-)$',
               'pos': '$(+)$',
               'direct': 'super(set); setdiff; neg-absent',
               'mirror': 'subset; pos-present'}

    env_info = target_am[['approx', 'selection', 'l1']].assign(
        approx=(target_am.approx + '^[aliases: ' +
                target_am.approx.map(aliases) + ']'),
        env_key=target_am.index.str.removesuffix('∩*exactly*'),
        polarity=target_am.polarity + ' ' + target_am.polarity.map(aliases),
        directory=target_am.l1.map(dir_names),
        aliases=target_am.l1.map(aliases),
    )
    env_info = env_info.rename(columns={
        'approx': 'polarity approximation method',
        'selection': '(positive) token selection',
        'env_key': 'environment key',
        'directory': 'processing directory'})

    env_info.to_markdown(env_info_path, index=None)

In [62]:
N_vals = target_am.N
normed_counts = target_am[['f','f1', 'f2']].apply(lambda r: r/N_vals[r.name] , axis=1).multiply(100).rename(
    columns={'f': f'E ∩ *{TARGET}*', 
             'f1':'E ∩ $*$', 
             'f2':f'$*$ ∩ *{TARGET}* '}
    )
normed_counts.columns = r'%N: ' + normed_counts.columns.astype('string')
# nb_show_table(target_am[['approx', 'selection', 'polarity']].join(normed_counts), n_dec=2, 
#               outpath=cobble_dated_path(data_dir=TARGET_OUT, undated_stem=f'compare-envs~{TARGET}_percents', suffix='.md'))
env_info.join(normed_counts).to_markdown(
    cobble_dated_path(data_dir=TARGET_OUT, suffix='.md', 
                      undated_stem=f'compare-envs~{TARGET}_percents'), floatfmt=',.2f', intfmt=',')

In [63]:
target_am

,approx,selection,polarity,f,dP1,LRC,P1,G2,l1,l2,...,f2,N,exp_f,unexp_f,unexp_r,dP2,P2,odds_r_disc,t,MI
comparison,,,,,,,,,,,,,,,,,,,,,
ALL+: NEGany∩*exactly*,direct,ALL+,neg,43813,0.737327,6.190841,0.780855,216169.97875,NEGATED,exactly,...,56109,71961373,2474.55072,41338.44928,0.94352,0.013626,0.013805,1.893734,197.493439,1.248107
ALL+: COM∩*exactly*,direct,ALL+,pos,12296,-0.737327,-6.190841,0.219145,-216169.97875,COMPLEMENT,exactly,...,56109,71961373,53634.44928,-41338.44928,-3.361943,-0.013626,0.000179,-1.893734,-372.796875,-0.63968
NEQ+: NEGany∩*exactly*,direct,NEQ+,neg,43813,0.490699,5.970259,0.987268,55763.99325,NEGATED,exactly,...,44378,6347362,22189.0,21624.0,0.493552,0.013627,0.013805,1.895135,103.308136,0.295465
NEQ+: COM∩*exactly*,direct,NEQ+,pos,565,-0.490699,-5.970259,0.012732,-55763.99325,COMPLEMENT,exactly,...,44378,6347362,22189.0,-21624.0,-38.272568,-0.013627,0.000178,-1.895135,-909.728516,-1.594089
ALL+: NEGmir∩*exactly*,mirror,ALL+,neg,813,0.615346,3.60849,0.788555,1868.339071,NEGMIR,exactly,...,1031,1680633,178.96756,634.03241,0.779868,0.00263,0.002787,1.249731,22.236488,0.657316
ALL+: POS∩*exactly*,mirror,ALL+,pos,218,-0.615346,-3.60849,0.211445,-1868.339071,POSMIR,exactly,...,1031,1680633,852.03244,-634.03241,-2.908406,-0.00263,0.000157,-1.249731,-42.942097,-0.592
NEQ+: NEGmir∩*exactly*,mirror,NEQ+,neg,813,0.443811,3.087807,0.943155,819.818665,NEGMIR,exactly,...,862,583470,431.0,382.0,0.469865,0.002619,0.002787,1.216891,13.397325,0.275613
NEQ+: POS∩*exactly*,mirror,NEQ+,pos,49,-0.443811,-3.087807,0.056845,-819.818665,POSMIR,exactly,...,862,583470,431.0,-382.0,-7.795918,-0.002619,0.000168,-1.216891,-54.57143,-0.944281


In [64]:
target_count_out = (target_am.copy()
                    .filter(['approx', 'polarity', 'selection', 'N', 'f', 'f1', 'f2', 'exp_f', 'unexp_f'])
                    .rename(columns={'f': f'E ∩ *{TARGET}* (f)',
                                     'f1': 'E ∩ $*$ (f1)',
                                     'f2': f'$*$ ∩ *{TARGET}* (f2)',
                                     'N': f'selection total (N)',
                                     'exp_f': 'expected f',
                                     'unexp_f': 'observed f deviation'}
                            ).round())
# target_count_out.index = (target_count_out.index
#                           .str.replace(r'(?<!\*)\b(?![\*\+])', ' ', regex=True)
#                           .str.replace('~', '&')
#                           .str.replace(TARGET, f'*{TARGET}*'))
target_count_out.to_markdown(
    cobble_dated_path(data_dir=TARGET_OUT, suffix='.md',
                      undated_stem=f'compare-envs~{TARGET}_counts'),
    floatfmt=',.0f', intfmt=',')

# nb_show_table(target_am.filter(['approx', 'polarity','selection',  'f', 'N', 'f2', 'f1']).rename(
#     columns={'f':f'joint_ENV~{TARGET}',
#              'f1':'E margin',
#              'f2': f'*{TARGET}* margin',
#              'N': 'compare_total'}),
#               adjust_columns=False,
#               outpath= cobble_dated_path(data_dir=TARGET_OUT,
#                                          undated_stem=f'compare-envs~{TARGET}_counts',
#                                          suffix='.md'))
#   outpath=TARGET_OUT/f'compare-envs~{TARGET}_counts.md')

| compare_key              | approx   | polarity   | selection   |   N = selection total |   f = E & *exactly* |   f1 = E & $*$ |   f2 = $*$ & *exactly* |
|:-------------------------|:---------|:-----------|:------------|----------------------:|--------------------:|---------------:|-----------------------:|
| ALL+: NEGany & *exactly* | direct   | neg        | ALL+        |            71,961,373 |              43,813 |      3,173,681 |                 56,109 |
| ALL+: COM & *exactly*    | direct   | pos        | ALL+        |            71,961,373 |              12,296 |     68,787,692 |                 56,109 |
| NEQ+: NEGany & *exactly* | direct   | neg        | NEQ+        |             6,347,362 |              43,813 |      3,173,681 |                 44,378 |
| NEQ+: COM & *exactly*    | direct   | pos        | NEQ+        |             6,347,362 |                 565 |      3,173,681 |                 44,378 |
| ALL+: NEGmir & *exactly* | mirror   | neg        | ALL+        |             1,680,633 |                 813 |        291,735 |                  1,031 |
| ALL+: POS & *exactly*    | mirror   | pos        | ALL+        |             1,680,633 |                 218 |      1,388,898 |                  1,031 |
| NEQ+: NEGmir & *exactly* | mirror   | neg        | NEQ+        |               583,470 |                 813 |        291,735 |                    862 |
| NEQ+: POS & *exactly*    | mirror   | pos        | NEQ+        |               583,470 |                  49 |        291,735 |                    862 |


In [65]:
nb_show_table(
    target_am.filter(regex=r'^[^fuel]')
    # .join(
    #     target_am.filter(like='P').multiply(100).round(1),
    #     rsuffix='%')
    ,n_dec=3, 
    outpath=cobble_dated_path(data_dir=TARGET_OUT, 
                              undated_stem=f'compare-envs~{TARGET}_{TOKEN_TYPE}-am-scores', 
                              suffix='.md')
)


|                            | `approx`   | `selection`   | `polarity`   |   `dP1` |   `LRC` |   `P1` |         `G2` |        `N` |   `dP2` |   `P2` |   `odds_r_disc` |      `t` |   `MI` |
|:---------------------------|:-----------|:--------------|:-------------|--------:|--------:|-------:|-------------:|-----------:|--------:|-------:|----------------:|---------:|-------:|
| **ALL+: NEGany∩*exactly*** | direct     | ALL+          | neg          |   0.737 |   6.191 |  0.781 |  216,169.979 | 71,961,373 |   0.014 |  0.014 |           1.894 |  197.493 |  1.248 |
| **ALL+: COM∩*exactly***    | direct     | ALL+          | pos          |  -0.737 |  -6.191 |  0.219 | -216,169.979 | 71,961,373 |  -0.014 |  0.000 |          -1.894 | -372.797 | -0.640 |
| **NEQ+: NEGany∩*exactly*** | direct     | NEQ+          | neg          |   0.491 |   5.970 |  0.987 |   55,763.993 |  6,347,362 |   0.014 |  0.014 |           1.895 |  103.308 |  0.295 |
| **NEQ+: COM∩*exactly***    | direct     | NEQ+      

In [66]:
# nb_show_table(
#     target_am.filter(regex=r'^[^fuel]')
#     # .join(
#     #     target_am.filter(like='P').multiply(100).round(1),
#     #     rsuffix='%')
#     ,n_dec=3,
#     outpath=cobble_dated_path(data_dir=TARGET_OUT,
#                               undated_stem=f'compare-envs~{TARGET}_{TOKEN_TYPE}-am-scores',
#                               suffix='.md')
# )

targ_am_out = target_am.filter(regex=r'^[^fuel]|^f$')
# table_1 = targ_am_out.filter(regex=r'^[^dP]')
# table_1 = targ_am_out.filter(regex=r'^[^dP]').to_markdown(
#     floatfmt=',.3f', intfmt=',')

cobble_dated_path(
    data_dir=TARGET_OUT,
    undated_stem=f'compare-envs~{TARGET}_{TOKEN_TYPE}-am-scores_split',
    suffix='.md'
).write_text(
    "<!-- markdownlint-disable-file first-line-heading-->\n\n"
    + "Symmetric AM\n\n"
    + targ_am_out.filter(['approx', 'selection','N', 'polarity','f', 'LRC', 'G2','odds_r_disc', 't','MI']
                         ).rename(columns={'odds_r_disc': 'odds ratio'}).to_markdown(floatfmt=',.3f', intfmt=',')
    + "\n\nAsymmetric AM\n\n"
    + targ_am_out.filter(['approx', 'selection','N', 'polarity','f','dP1','P1','dP2','P2']
                         ).to_markdown(floatfmt=',.3f', intfmt=',')
)

2753

In [67]:
parq_paths = list(TOP_AM_DIR.joinpath('ALL/ALL-Top8').glob('*.parq'))
parq_paths.sort()
load_path = parq_paths[-1]
type_form_col = 'bigram_lower' if TOKEN_TYPE=='bigram' else f'{TOKEN_TYPE}_form_lower'
hits = pd.read_parquet(load_path, engine='pyarrow', filters=[(type_form_col, '==', TARGET)])

In [68]:
other_neg_hits = catify(hits[~(hits.neg_lemma.isin({'', 'not'}))].drop_duplicates(subset=['text_window']), reverse=True)
nb_show_table(other_neg_hits.value_counts('neg_lemma').to_frame('#_in_sample').reset_index())


|        | `neg_lemma`   |   `#_in_sample` |
|:-------|:--------------|----------------:|
| **1**  | never         |              93 |
| **2**  | none          |              54 |
| **3**  | neither       |              20 |
| **4**  | ain't         |              19 |
| **5**  | nobody        |              19 |
| **6**  | nor           |              17 |
| **7**  | nothing       |              16 |
| **8**  | no            |               8 |
| **9**  | aint          |               2 |
| **10** | few           |               2 |
| **11** | without       |               2 |



### Pull examples of non-"not" lemma triggers

In [69]:
n_ex = 3

formatted_token_str = (
    other_neg_hits[['token_str']]
    .apply(
        embolden,
        mono=False,
        bold_regex=(
            r'\b(' + TARGET.replace('_', ' ') + r' \w+)\b'))
    .apply(
        embolden,
        bold_regex=(
            r'\b('+r'|'.join(other_neg_hits.neg_lemma.unique())+r')\b'))
    .apply(italic)
    .squeeze()).str.replace('__ __', ' ')

formatted_hits = other_neg_hits.assign(
    token_str=formatted_token_str)
formatted_sample = (
    pd.concat(
        [d.sample(min(len(d), n_ex)) for __, d
         in formatted_hits.drop_duplicates('all_forms_lower').groupby('neg_lemma', observed=True)])
    .sort_values(['all_forms_lower'])
    .reset_index())

nb_show_table(formatted_sample
              .filter(['neg_lemma', 'adj_form_lower', 'token_str', 'hit_id'])
              .assign(hit_id='`'+formatted_sample.hit_id.astype('string')+'`'),
              outpath=cobble_dated_path(
                  data_dir=TARGET_OUT, undated_stem=f'other-neg-lemmas_{TARGET}-ex', suffix='.md')
              )


|        | `neg_lemma`   | `adj_form_lower`   | `token_str`                                                                                                                                                                                                                                                            | `hit_id`                                    |
|:-------|:--------------|:-------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:--------------------------------------------|
| **1**  | ain't         | free               | *If you do n't follow all of these items , you can consider your butt sued -- and that __``ain't`` exactly free__ .*                                                                                                                     